In [82]:
import os
import pytesseract
from pdf2image import convert_from_path
import requests
import json
from sentence_transformers import SentenceTransformer
import re
import weaviate
import weaviate.classes as wvc
from weaviate.classes.config import Property, DataType
import ollama
from weaviate.util import generate_uuid5
import subprocess

In [83]:

# Initialize SentenceTransformer model
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

# Set directory containing PDFs
pdf_directory = "C:/Users/AbhinavKasubojula/OneDrive - Kenall Inc/Desktop/code/docs/"

llm = "llama3.2"


In [84]:


# Compute embeddings for documents
def compute_embeddings(documents):
    embeddings = model.encode(documents, convert_to_tensor=False)
    return embeddings


In [85]:

# Extract text from a PDF using OCR
def extract_text_from_pdf(pdf_path):
    try:
        images = convert_from_path(pdf_path)  # Convert PDF to images
        text = ""
        for page_num, image in enumerate(images):
            text += pytesseract.image_to_string(image)  # OCR on the image
            print(f"Extracted text from page {page_num + 1} of {os.path.basename(pdf_path)}...")
        return text
    except Exception as e:
        print(f"Error extracting text from {pdf_path}: {e}")
        return None


In [86]:
import re

def text_to_dict(text):
    result = {}
    current_key = None
    
    # Split text into lines
    lines = text.strip().split("\n")
    
    # Check if the first line looks like an unwanted heading
    if re.match(r"^(Here is a summary|Summary of)", lines[0], re.IGNORECASE):
        lines = lines[1:]  # Skip the first line if it's an unwanted heading

    for line in lines:
        if ":" in line:  # Line contains a key-value pair
            key, value = line.split(":", 1)
            current_key = key.strip()
            result[current_key] = value.strip()
        else:
            if current_key:  # Append to the previous key's value
                result[current_key] += " " + line.strip()
    
    return result


In [87]:
def summarize_text_with_llm(text):
    prompt = f"""
    You are an expert in summarizing and structuring resumes. Your task is to extract information and organize it into the exact structure below. The output format must not change across different resumes, regardless of the content.

    Rules:
    Use the exact headings listed below without any modification or omission.
    Leave headings blank if no information is available for that section.
    Do not rearrange, rename, or skip any headings.
    Licenses must appear only under the "Licenses" heading.
    Extract company names consistently, ensuring accuracy.
    Output Format (retain these exact headings):

     Name: [Name here]  
     Education Qualification: [Qualifications here, e.g., PE, BA, etc.]  
     Role: [Role here]  
     Licenses: [Licenses here]  
     Company: [Company names here]  
     Qualifications: [Additional qualifications here]  
     Experience: [Experience details here]  
     Projects he/she worked on: [Projects listed here]  
     Roles and Responsibilities: [Responsibilities here]  

    Resume Text:
    {text}
    Output Instructions:
    Follow the exact structure provided above.
    Include all information under the correct headings.
    Do not modify or skip any section, even if a section is blank.

    """
    result = subprocess.run(
        ["ollama", "run", llm],
        input=prompt,
        text=True,
        capture_output=True
    )

    return result.stdout


In [88]:
def remove_asterisks(text):
    # Remove all occurrences of single and double asterisks
    clean_text = re.sub(r"\*{1,2}", "", text)
    return clean_text

In [89]:

# Process each PDF, extract text, summarize, and store in Weaviate
def process(sum_dict, emb, summary_text):
    pdf_files = [f for f in os.listdir(pdf_directory) if f.endswith(".pdf")]
    for pdf_file in pdf_files:
        pdf_path = os.path.join(pdf_directory, pdf_file)
        print(f"Processing: {pdf_file}")
        
        # Extract text from the PDF
        text = extract_text_from_pdf(pdf_path)
        if text:
            # Summarize the extracted text
            summary1 = summarize_text_with_llm(text)
            #print(f"Summary of {file_name}: {summary[:100]}...")  # Print first 100 characters of summary
            summary = remove_asterisks(summary1)
            summary_dict= text_to_dict(summary)
            print(summary_dict)
            

            # Generate vector embedding
            embedding = compute_embeddings([summary1])[0]           
            # Store data in Weaviate
            #store_in_weaviate(summary, embedding,collection_name)
        sum_dict.append(summary_dict)
        emb.append(embedding)
        summary_text.append(summary)

    return(sum_dict, emb, summary_text)

# Run the process
sum_dict=[]
embeddings =[]
summary_text =[]
sum_dict, embeddings, summary_text = process(sum_dict, embeddings, summary_text)


Processing: page_1 (2).pdf
Extracted text from page 1 of page_1 (2).pdf...
{'Name': 'Kris D. Prasad', 'Education Qualification': 'M.S., Civil Engineering, 1992; P.E. (Civil): TX #91952, LA #34186, NM #20737, PR #28316', 'Role': 'Program/Project Manager', 'Licenses': 'Secret Clearance', 'Company': 'Kenall-Hana JV, LLC | Houston, TX', 'Qualifications': 'Organizations: SAME, ASCE, ACEC, College of Engineers and Surveyors of Puerto Rico', 'Experience': 'a. TOTAL: 29; b. WITH CURRENT FIRM: 20', 'Projects he/she worked on': '', '1. USACE Fort Worth District - Renovation of Buildings 16, 44, 615, 2002, and 2006, Fort Sam Houston, TX construction': '2022', 'Brief Description': 'Program/Project Manager for the v Oversaw site investigation and reviewed seismic site evaluation as per preparation of design package for ASCE 31. ', '2. USACE Fort Worth District - Repair Fire Protection, Install Security Fences and Repairs of Buildings at Red River Army Depot, Texarkana, TX construction': '2022', '4.

Exception in thread Thread-648 (_readerthread):
Traceback (most recent call last):
  File "C:\Users\AbhinavKasubojula\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "c:\Users\AbhinavKasubojula\OneDrive - Kenall Inc\Desktop\code\.venv\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\AbhinavKasubojula\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\AbhinavKasubojula\AppData\Local\Programs\Python\Python312\Lib\subprocess.py", line 1599, in _readerthread
    buffer.append(fh.read())
                  ^^^^^^^^^
  File "C:\Users\AbhinavKasubojula\AppData\Local\Programs\Python\Python312\Lib\encodings\cp1252.py", line 23, in decode
    return codecs.charmap_decode(input,self.errors,decoding_table)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Unicode

{'Name': 'Kris D. Prasad', 'Education Qualification': 'MS, Civil Engineering, 1992; PE (Civil): TX #91952 (2003), LA #34186, NM #20737, PR# 28316', 'Role': 'Program Manager/Senior Structural Engineer', 'Licenses': 'PE (Civil); TX #91952 (2003), LA #34186, NM #20737, PR# 28316', 'Company': 'SAT-Kenall JV | Houston, TX', 'Qualifications': 'Additional qualifications not provided in the text', 'Experience': 'Over 29 years of professional engineering and management experience with various federal, state, municipal, and local programs. Currently serving as Program Manager for USACE Fort Worth and Tulsa Districts on various civil works and military projects that included new construction, renovation, and improvements of DoD facilities. Previously served as task order manager for two (2) NAVFAC Southeast projects in Meridian Base, which received excellent ratings for all criteria. Experience includes 1000 projects with a value of over $100M. Organizations: SAME, ASCE, ACEC, College of Engineer

Exception in thread Thread-688 (_readerthread):
Traceback (most recent call last):
  File "C:\Users\AbhinavKasubojula\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "c:\Users\AbhinavKasubojula\OneDrive - Kenall Inc\Desktop\code\.venv\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\AbhinavKasubojula\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\AbhinavKasubojula\AppData\Local\Programs\Python\Python312\Lib\subprocess.py", line 1599, in _readerthread
    buffer.append(fh.read())
                  ^^^^^^^^^
  File "C:\Users\AbhinavKasubojula\AppData\Local\Programs\Python\Python312\Lib\encodings\cp1252.py", line 23, in decode
    return codecs.charmap_decode(input,self.errors,decoding_table)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
Unicode

{'Name': 'Robert Gaylord ', 'Education Qualification': 'BS, Civil Engineering, 1987; PE (Civil): FL #51373 (1997); Other Professional Qualifications: Publications, Organizations, Training, Awards, etc. ', 'Role': 'Sr. Civil Engineer ', 'Licenses': 'None listed ', 'Company': 'SAI-Kenall JV | Orlando, FL ', 'Qualifications': 'Additional qualifications not provided in the text ', 'Experience': '35 years of experience conducting engineering design, analysis, and developing technical specifications for civil infrastructure design projects. Led and supported projects for various USACE Districts, US Air Force, and other federal agencies including military installations and federal facilities in Florida and Puerto Rico. Expertise encompasses a wide variety of civil infrastructure design including site design, utilities, roads, parking, airfields, parks, pump stations, and stormwater management. 30-year expertise in complex H&H modeling and flood control design as well as administering construc

In [90]:
#
for i in range(len(sum_dict)):
    sum_dict[i]["vector"] = embeddings[i].tolist()
    sum_dict[i]["summary"] = summary_text[i]

In [91]:
len(sum_dict)

6

In [92]:
from weaviate.classes.config import ReferenceProperty
client = weaviate.connect_to_local(port=8082)
# Delete a collection
employee = "employee"
experience = "experience"
#client.collections.delete(name=collection_name)
#print(f"Collection '{collection_name}' deleted successfully.")

if client.collections.exists(name="experience"):
    client.collections.delete(name="experience")
    print("Existing", experience ,"collection deleted.")

client.collections.create(
    name=experience,  # Name of the data collection
    properties=[
        Property(name="Name", data_type=DataType.TEXT),
        Property(name="emp_id", data_type=DataType.TEXT),
        Property(name="role", data_type=DataType.TEXT),
        Property(name="company", data_type=DataType.TEXT),
        Property(name="qualification", data_type=DataType.TEXT),
        Property(name="experience", data_type=DataType.TEXT),
        Property(name="projects", data_type=DataType.TEXT),
        Property(name="roles_responsibilities", data_type=DataType.TEXT),
        Property(name="summary", data_type = DataType.TEXT)
    ])
print("Collection",experience," created successfully.")

if client.collections.exists(name=employee):
    client.collections.delete(name=employee)
    print("Existing",employee,"collection deleted.")


client.collections.create(
    name=employee,  # Name of the data collection
    properties=[
        Property(name="Name", data_type=DataType.TEXT,index_searchable=True,index_filterable=True),
        Property(name="role", data_type=DataType.TEXT,index_searchable=True,index_filterable=True),
        Property(name="company", data_type=DataType.TEXT,index_searchable=True,index_filterable=True),
        Property(name="qualification", data_type=DataType.TEXT,index_searchable=True,index_filterable=True),
        Property(name="experience", data_type=DataType.TEXT,index_searchable=True,index_filterable=True),
        Property(name="projects", data_type=DataType.TEXT,index_searchable=True,index_filterable=True),
        Property(name="roles_responsibilities", data_type=DataType.TEXT,index_searchable=True,index_filterable=True),
        Property(name="summary", data_type = DataType.TEXT,index_searchable=True,index_filterable=True)
    ],
    references=[
        ReferenceProperty(
            name="exp_id",
            target_collection=experience
        )
    ]

)
print("Collection",employee," created successfully.")

if client.collections.exists(name="dup_coll"):
    client.collections.delete(name="dup_coll")
    print("Existing collection deleted.")


client.collections.create(
    name="dup_coll",  # Name of the data collection
    properties=[
        Property(name="Name", data_type=DataType.TEXT,index_searchable=True,index_filterable=True),
        Property(name="role", data_type=DataType.TEXT,index_searchable=True,index_filterable=True),
        Property(name="company", data_type=DataType.TEXT,index_searchable=True,index_filterable=True),
        Property(name="qualification", data_type=DataType.TEXT,index_searchable=True,index_filterable=True),
        Property(name="experience", data_type=DataType.TEXT,index_searchable=True,index_filterable=True),
        Property(name="projects", data_type=DataType.TEXT,index_searchable=True,index_filterable=True),
        Property(name="roles_responsibilities", data_type=DataType.TEXT,index_searchable=True,index_filterable=True),
        Property(name="summary", data_type = DataType.TEXT,index_searchable=True,index_filterable=True)
    ]

)
print("collection dup_coll created")
client.close()

Existing experience collection deleted.
Collection experience  created successfully.
Existing employee collection deleted.
Collection employee  created successfully.
Existing collection deleted.
collection dup_coll created


In [99]:
sum_dict[4]

{'Name': 'Robert Morris',
 'Education Qualification': '',
 'Licenses': 'Registered Architect: TX #12112 (1987), NM, Registered Interior Designer: TX # 1985 (1993)',
 'Company': 'SAI-Kenall JV | Houston, TX',
 'Qualifications': '',
 'Experience': '',
 'Projects he/she worked on': '',
 'Roles and Responsibilities': ' ',
 '(1) TITLE AND LOCATION (City and State)': '(2) YEAR COMPLETED',
 'USACE Fort Worth District - Renovation of Buildings 16, 44, 615, 2002, and _| proressiona services': '2022  Brief Description (Brief scope, size, cost, etc.) AND SPECIFIC ROLE DQC Manager for the development of | Y Prepared DQC plan and ensured quality of all deliverables. five (5) separate 100% plans and specs to Y Reviewed architectural designs including the renovation of build- renovate five historical buildings (total ings for additional office space, restrooms, new electrical/mechani- 100,400 SF renovation area) that range cal systems, new interior walls, ceilings, flooring, new fire protec- from 100

In [101]:
client = weaviate.connect_to_local(port=8082)

# Get the collections
emp = client.collections.get("employee")
exp = client.collections.get("experience")
dup = client.collections.get("dup_coll")

# Batch process for the 'employee' collection
employee_names = []
employee_uuids ={}
dup_emp = {}
with emp.batch.dynamic() as batch:

    for d in sum_dict:
        if d["Name"] not in employee_names:
            employee_uuid = batch.add_object({

                "Name": d["Name"],
                "license":next(
                        (d.get(key) for key in ["Licenses/Certifications", "Licenses", "PE (Civil)"] if key in d),
                        "Not provided"),
                "role": next(
                        (d.get(key) for key in ["Role", "Roles and Responsibilities"] if key in d),
                        "Not provided"),
                "company": d["Company"],
                "qualification": d["Qualifications"],
                "experience": d["Experience"],
                "summary": d["summary"],
            }, vector=d["vector"])
            employee_names.append(d["Name"])
            employee_uuids[d["Name"]] = employee_uuid  # Map Name to UUID for later reference
        else:
            dup_emp[d["Name"]] = d["summary"]
for item in emp.iterator():
    if item.properties["name"] in dup_emp:
        print("updating ", item.properties["name"])
        emp.data.update(
            uuid=employee_uuids[item.properties["name"]],
            properties={
                "summary": dup_emp[item.properties["name"]],
            }
        )

client.close()


updating  Kris D. Prasad
updating  Kris D. Prasad


c:\Users\AbhinavKasubojula\OneDrive - Kenall Inc\Desktop\code\.venv\Lib\site-packages\weaviate\warnings.py:314: ResourceWarning: Con004: The connection to Weaviate was not closed properly. This can lead to memory leaks.
            Please make sure to close the connection using `client.close()`.
  warnings.warn(


clear till here(data loaded in Db)

In [103]:
client = weaviate.connect_to_local(port=8082)
emp = client.collections.get("employee")
query_vector = compute_embeddings("get who worked as civil engineer")

response = emp.query.near_vector(
    near_vector=query_vector,
    limit=2,
    return_metadata=wvc.query.MetadataQuery(certainty=True)
)

print(response.objects[0].properties["summary"])

Name: Robert Gaylord  

Education Qualification: BS, Civil Engineering, 1987; PE (Civil): FL #51373 (1997); Other Professional Qualifications: Publications, Organizations, Training, Awards, etc. 

Role: Sr. Civil Engineer  

Licenses: None listed  

Company: SAI-Kenall JV | Orlando, FL  

Qualifications: Additional qualifications not provided in the text  

Experience: 35 years of experience conducting engineering design, analysis, and developing technical specifications for civil infrastructure design projects. Led and supported projects for various USACE Districts, US Air Force, and other federal agencies including military installations and federal facilities in Florida and Puerto Rico. Expertise encompasses a wide variety of civil infrastructure design including site design, utilities, roads, parking, airfields, parks, pump stations, and stormwater management. 30-year expertise in complex H&H modeling and flood control design as well as administering construction & implementation o

In [105]:
client = weaviate.connect_to_local(port=8082)
collection = client.collections.get("employee")

for item in collection.iterator():
    print(item.uuid, item.properties["summary"])

04b95b48-1eb9-485e-9ec3-bf16f7deecc7 Name: Brett Witte  

Education Qualification: BS, Civil Engineering (2005), PE (Civil): FL #70720 (2010)  

Role: Project Manager / Civil Engineer  

Licenses:  PE (Civil): FL #70720 (2010)

Company: SAI-Kenall JV | Orlando, FL, USACE Fort Worth District  

Qualifications: Additional qualifications not available

Experience:
- Brett Witte, PE
  a. Total: 18 years 
  b. WITH CURRENT FIRM: 18 years 

Project Details:
- Paint and Sandblast Facility, USCG Miami Station, FL (2021)
  (1) TITLE AND LOCATION (City and State): Paint and Sandblast Facility, USCG Miami Station, FL
  (2) YEAR COMPLETED: 2021
  (3) BRIEF DESCRIPTION (Brief scope, size, cost, etc.) AND SPECIFIC ROLE XX
    Project Manager / Civil Engineer. The pro- Y Responsible for coordinating between disciplines.
  (4) PROJECT DESCRIPTION
    Project consisted of the construction of a new paint Y Led civil design (utilities, layout, grading, drainage)
and sandblast facility to replace the exis